In [1]:
import cvxopt as cvo
import numpy as np
import math

In [2]:
class PLA:
    def __init__(self, thresh):
        #threshold for pct wrong, for guaranteed linearly seperable, set to 0
        self.thresh = max(0, thresh)
        #since we don't want sit here forever, let's stick an upper limit to training rounds
        self.limit = 10e5
        

    def reset_weights(self):
        self.weights = np.zeros(1+self.dim) #adding one for w0

    def set_dim(self, dim):
        self.dim = max(1, dim)
        self.reset_weights()
        self.n_iter = 0 #number of training rounds
        
    def X_reshape(self,X):
        if len(X.shape) <= 1:
            return np.r_[[1.0], X]
        num_examples = X.shape[0]
        real_X = np.c_[np.ones(num_examples), X]
        return real_X

    def predict(self,X):
        real_X = self.X_reshape(X)
        cur_h = np.dot(real_X, self.weights)
        return cur_h
            

    def calc_error(self, X,Y):
        num_ex = X.shape[0]
        predicted = np.sign(self.predict(X))
        num_incorrect = np.sum(np.not_equal(predicted, np.sign(Y)))
        prop_incorrect = float(num_incorrect)/float(num_ex)
        return prop_incorrect

    def train(self,X,Y):
        if len(X.shape) <= 1:
            dim = X.shape[0]
            X = self.X_reshape((1,dim))
        self.set_dim(X.shape[1])
        num_ex = X.shape[0]
        ex_idxs = np.arange(num_ex) #indexing the number of examples and shuffling
        under_thresh = False
        n_iter = 0 #number of iterations
        while under_thresh == False:
            np.random.shuffle(ex_idxs)
            num_wrong = 0
            for idx in ex_idxs:
                guess = self.predict(X[idx])
                real_ex = np.r_[[1.0], X[idx]] #current example with x0
                agreed = np.sign(guess) == Y[idx]
                if not agreed:
                    self.weights = self.weights + np.multiply(Y[idx], real_ex)
                    num_wrong = num_wrong + 1
            pct_wrong = float(num_wrong)/float(num_ex)
            under_thresh = pct_wrong <= self.thresh
            n_iter = n_iter + 1
            if n_iter >= self.limit:
                break
        self.n_iter = n_iter

In [3]:
class SVM:
    def __init__(self):
        self.thresh = 1.0e-5
        #suppress output
        cvo.solvers.options['show_progress'] = False

    def kernel_calc(self, X):
        #kernel calculation
        return np.dot(X,X.T)

    def get_constraints(self, num_ex):
        #make constraints matrices G, h being passed number of examples
        #-alphas <= 0
        G = cvo.matrix(np.multiply(-1, np.eye(num_ex)))
        # h = 0
        h = cvo.matrix(np.zeros(num_ex))
        return G, h

    def X_reshape(self,X):
        num_examples = X.shape[0]
        real_X = np.c_[np.ones(num_examples), X]
        return real_X

    def calc_error(self, X,Y):
        num_ex = X.shape[0]
        predicted = np.sign(self.predict(X))
        num_incorrect = np.sum(np.not_equal(predicted, np.sign(Y)))
        prop_incorrect = float(num_incorrect)/float(num_ex)
        return prop_incorrect

    def predict(self,X):
        real_X = self.X_reshape(X)
        cur_h = np.matmul(real_X, self.weights)
        return cur_h

    def train(self,X,Y):
        #expecting X as Nxd matrix and Y as a Nx1 matrix
        #note: no reshaping for X
        X = X.astype(float)
        Y = Y.astype(float)
        num_ex, cur_dim = X.shape
        q = cvo.matrix(np.multiply(-1, np.ones((num_ex,1))))
        P = cvo.matrix(np.multiply(np.outer(Y, Y), self.kernel_calc(X)))
        A = cvo.matrix(Y.reshape(1, num_ex), tc='d')
        b = cvo.matrix(0.0)
        G, h = self.get_constraints(num_ex)
        cvo_sol = cvo.solvers.qp(P,q,G,h,A,b)
        alphas = np.ravel(cvo_sol['x'])
        #now to find the weight vector = sum(i=1,N) an*yn*xn
        yx = np.multiply(Y.reshape((num_ex, 1)),X)
        weights= np.sum(np.multiply(alphas.reshape(num_ex,1), yx), axis=0)
        #now we want to find the w0 term so pick an sv and solve
        #yn(wTxn + b) = 1
        #-> 1/yn = wTxn + b, (1/yn)-wTxn = b
        #find the sv with the corresponding alpha bigger than thresh
        alphas_thresh = np.greater_equal(alphas,self.thresh)
        sv_idx = np.argmax(alphas_thresh)
        wtxn = np.dot(weights, X[sv_idx])
        cur_b = (1.0/Y[sv_idx]) - wtxn
        self.weights = np.concatenate(([cur_b], weights))
        self.alphas = alphas
        self.num_alphas = np.sum(alphas_thresh)

In [4]:
class Line:
    def __init__(self, p1, p2):
        #input: 2 2-dim numpy arrays
        self.p1 = p1
        self.p2 = p2
        diff = np.subtract(p2, p1)
        self.slope = diff[1]/diff[0]
        self.is_vert = False
        #point slope form = y - y1 = m(x - x1) 
        #y = 0 -> -y1 = m(x - x1) -> -y1/m = x - x1 -> (-y1/m) + x1 = x
        self.y_int = ((-1 * p1[1])/self.slope) + p1[0]

        
    def calc(self,testpt):
        #input: numpy array with 2 dim
        
        if self.is_vert == False:
            line_y = self.slope*testpt[0] + self.y_int
            diff = testpt[1] - line_y
        else:
            line_x = self.p1[0]
            diff = testpt[0] - line_x
        return np.sign(diff)

    def calc_pts(self,testpts):
        #testpts should be Nx2
        #goal: test against equation of line, if above, then return +1
        #if on line 0, else -1
        #to check:
        #slope-intercept: y = mx + b or (y-b)/m = x
        if len(testpts.shape) <= 1:
            testpts = testpts.reshape((1,2))
        line_y = np.add(self.y_int, np.multiply(testpts[:,0], self.slope))
        diff = np.subtract(testpts[:,1], line_y)
        return np.sign(diff)

class LineTest:
    def __init__(self, num_train, num_test):
        self.thresh = 0.001
        line_pts = np.zeros((2,2))
        #don't want line vertical or near vertical
        while abs(line_pts[0][1] - line_pts[1][1]) <= self.thresh:
            line_pts = np.random.uniform(-1,1, (2,2))
        self.line = Line(line_pts[0], line_pts[1])
        num_train = max(1, num_train)
        self.num_train = num_train
        self.train_set = np.random.uniform(-1,1, (num_train, 2))
        self.train_labels = self.line.calc_pts(self.train_set)
        num_test = max(1,num_test)
        self.num_test = num_test
        self.test_set = np.random.uniform(-1,1, (num_test, 2))
        self.test_labels = self.line.calc_pts(self.test_set)

In [9]:
num_exp = [1000,1000]
n_vals = [10,100]
n_test = 1000 #testing sample size
for idx,n in enumerate(n_vals):
    eout_pla = np.array([])
    eout_svm = np.array([])
    sv_svm = np.array([])
    cur_exp = num_exp[idx]
    my_pla = PLA(0) #threshold for number wrong since linearly separable
    my_svm = SVM()
    for my_exp in range(cur_exp):
        my_test = LineTest(n,n_test)
        my_pla.train(my_test.train_set,my_test.train_labels)
        my_svm.train(my_test.train_set, my_test.train_labels)
        pla_err = my_pla.calc_error(my_test.test_set, my_test.test_labels)
        svm_err = my_svm.calc_error(my_test.test_set, my_test.test_labels)
        n_sv = my_svm.num_alphas
        cur_adv = svm_err < pla_err
        eout_pla = np.concatenate((eout_pla, [pla_err]))
        eout_svm = np.concatenate((eout_svm, [svm_err]))
        sv_svm = np.concatenate((sv_svm, [n_sv]))
    eavg_pla = np.average(eout_pla)
    eavg_svm = np.average(eout_svm)
    svm_adv = 100.0*np.average(np.less_equal(eout_svm, eout_pla))
    sv_avg = np.average(sv_svm)
    print("=== For a training set with N = %d with %d runs ===" % (n, cur_exp))
    print("E_out avg for PLA: %f" % eavg_pla)
    print("E_out avg for SVM: %f" % eavg_svm)
    print("SVM improves on performance over PLA %f%% of the time" % svm_adv)
    print("Avg number of support vectors: %f" % sv_avg)

=== For a training set with N = 10 with 1000 runs ===
E_out avg for PLA: 0.087766
E_out avg for SVM: 0.076935
SVM improves on performance over PLA 70.900000% of the time
Avg number of support vectors: 2.165000
=== For a training set with N = 100 with 1000 runs ===
E_out avg for PLA: 0.011379
E_out avg for SVM: 0.008824
SVM improves on performance over PLA 75.100000% of the time
Avg number of support vectors: 2.509000
